# Spark DataFrame Missing Data

In [31]:
from pyspark.sql import SparkSession

In [32]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [33]:
df = spark.read.csv('ContainsNull.csv',header=True, 
                    inferSchema=True)

In [34]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Caso 1.

Quitamos los registros que tengan datos Null.

In [35]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



### Caso 2.

Quitamos los registros que tengan datos Null pero con umbral. Los registros que tengan el umbral o mas nulos se eliminan.

In [36]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Caso 3.

Usando el parametro 'how'.

In [37]:
# how='any' es como se aplica el default de df.na.drop()
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [38]:
# how='all' elimina las filas que tengan TODOS los campos Nulos.
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Caso 4.

Usando el parametro 'subset'.

In [39]:
# subset=[lista de campos].
# Elimina los registros que tengan Null en los campos de la lista
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Caso 5.

Rellenando los elementos Nulos con na.fill

In [40]:
# 
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [41]:
df.na.fill('Relleno').show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|Relleno| null|
|emp3|Relleno|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



Puede verse como el 'Relleno' solo se aplica en valores de tipo string.

In [42]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



En este ultimo caso solo rellena con ceros los numericos que tengan Null.

In [43]:
# Por supuesto podemos ser mas precisos en la forma de rellenar
df.na.fill('Sin nombre', subset=['Name']).show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Sin nombre| null|
|emp3|Sin nombre|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [44]:
# Ahora queremos rellenos con valores medios u otros criterios

from pyspark.sql.functions import mean

In [45]:
# primero sacamos la media de ventas
valor_medio = df.select(mean(df['Sales'])).collect()
valor_medio

[Row(avg(Sales)=400.5)]

In [46]:
# Nos devuelve una lista, cuyo primer elemento es:
valor_medio[0]

Row(avg(Sales)=400.5)

In [47]:
valor_medio[0][0]

400.5

In [48]:
media_ventas = valor_medio[0][0]

In [49]:
# Entonces ahora podemos pasar un valor unico como media al metodo fill
df.na.fill(media_ventas,['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [50]:
# Se puede hacer en una linea, aunque la sintaxis es menos intuitiva.

df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

